In [4]:
!pip install wget
import wget
import pandas as pd
import gzip
import json
import re
import os
import sys
from gensim.parsing.preprocessing import remove_stopwords

import string
LANGUAGE = string.ascii_lowercase + string.punctuation + ' '

import nltk
from nltk import word_tokenize

from nltk.stem import WordNetLemmatizer
LEMMATIZER = WordNetLemmatizer()


# from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, RegexpStemmer 
# ps = PorterStemmer()
# # ps = SnowballStemmer(language='english')
# # ps = LancasterStemmer()
# # ps = RegexpStemmer('ing$|s$|e$|able$', min=4)

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
COMMON_ABBREV = {
    "411":"information",
    "af":"as fuck",
    "afaik":"as far as i know",
    "ama":"ask me anything",
    "asl":"age / sex / location",
    "b4":"before",
    "bae":"before anyone else",
    "b/c":"because",
    "bc":"because",
    "bff":"best friends forever",
    "brb":"be right back",
    "btaim":"be that as it may",
    "bts":"behind the scenes",
    "btw":"by the way",
    "dae":"does anyone know?",
    "dftba":"don’t forget to be awesome",
    "dyk":"did you know",
    "eli5":"explain like i am 5 ( years old )",
    "f2f":"face to face",
    "fbf":"flashback friday",
    "ffs":"for fuck’s sake",
    "fml":"fuck my life",
    "fomo":"fear of missing out",
    "ftfy":"fixed that for you",
    "ftw":"for the win",
    "futab":"feet up, take a break",
    "fwiw":"for what it is worth",
    "fyi":"for your information",
    "gg":"good game",
    "gr8":"great",
    "gtg":"got to go",
    "gtr":"got to run",
    "h/t":"hat tip",
    "hbd":"happy birthday",
    "hth":"here to help / happy to help",
    "hmb":"hit me back",
    "hmu":"hit me up",
    "ianad":"i am not a doctor",
    "ianal":"i am not a lawyer",
    "icymi":"in case you missed it",
    "idc":"i do not care",
    "idk":"i do not know",
    "ikr":"i know, right?",
    "ily":"i love you",
    "imho":"in my humble opinion",
    "imo":"in my opinion",
    "imy":"i miss you",
    "irl":"in real life",
    "iso":"in search of",
    "jk":"just kidding",
    "jtm":"just the messenger",
    "l8":"late",
    "lmao":"laughing my ass off",
    "lmk":"let me know",
    "lol":"laughing out loud",
    "mtfbwy":"may the force be with you",
    "myob":"mind your own business",
    "nbd":"no big deal",
    "nm":"not much",
    "nsfw":"not safe for work",
    "nvm":"nevermind",
    "nyt":"name your trade",
    "obv":"obviously",
    "oh":"overheard",
    "omg":"oh my god",
    "omw":"on my way",
    "orly":"oh really?",
    "pls":"please",
    "ppl":"people",
    "potd":"photo of the day",
    "psa":"public service announcement",
    "qotd":"quote of the day",
    "rn":"right now",
    "rofl":"rolling on the floor laughing",
    "srsly":"seriously",
    "smh":"shaking my head",
    "tbh":"to be honest",
    "tbt":"throwback thursday",
    "tfw":"that feeling when / the face when",
    "tgif":"thank god it’s friday",
    "thx":"thanks",
    "til":"today i learned",
    "tl;dr":"too long; did not read",
    "tmi":"too much information",
    "ty":"thank you",
    "wbu":"what about you?",
    "wbw":"wayback wednesday",
    "wfh":"working from home",
    "wtf":"what the fuck",
    "wyd":"what are you doing?",
    "yolo":"you only live once",
    "ysk":"you should know",
    "yw":"you are welcome"
 }

In [20]:
def download_file(url):
    """
    input : url to zip files of amazon customer reviews
            Eg: "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
    output : path to downloaded file
             Eg: '../data/dataset/sample_us.tsv'
    """
    # import pdb;pdb.set_trace()
    # path = '../data/dataset'
    # zip_path = path + '/' + url.split('/')[-1]
    zip_path = url.split('/')[-1]
    unzip_path = zip_path.replace(".gz",'')

    if not os.path.exists(zip_path): #zip file not present 
        if not os.path.exists(unzip_path):
            print("Downloading ZIP file becasue File does not exist")
            filename = wget.download(url)
            print(filename)
            print("Dowload Complete\n\nInitiating Decompression")
            tsv_path = unzip(filename)
            print(tsv_path)
            print("Decompression Complete")
            # return filename
        else:
            print("Zip File Already Downloaded and Decompressed")
            tsv_path = unzip_path
    else: # zip file present already
        if os.path.exists(unzip_path):
            print("Zip File Already Downloaded and Decompressed")
            tsv_path = unzip_path
        else:
            print("Zip File Already Downloaded.\n\nInitiating Decompression")
            tsv_path = unzip(zip_path)
            print(tsv_path)
            print("Decompression Complete")

    return tsv_path


In [7]:
def delete(filename):
    """
    input : filepath to be deleted
            Eg: ""
    output : None
    """
    if os.path.exists(filename):
        os.remove(filename)
    else:
        print("The file does not exist:\t" + filename)
    return None

In [18]:
def unzip(infile):
    """
    input : just the filename to be unzipped. 
            Accpeted file type : .gz
            Eg: ".gz"
    output : name of unzipped file
             Eg:
    """
    # infile = '../data/dataset/' + filename.split('/')[-1]
    if infile.endswith('.gz'):
        tofile = infile.replace('.gz','')

        with open(infile, 'rb') as inf, open(tofile, 'w', encoding='utf8') as tof:
            decom_str = gzip.decompress(inf.read()).decode('utf-8')
            tof.write(decom_str)
        delete(infile)
        return tofile
    else:
        return infile

In [13]:
def load_dataset(filename):
    """
    input : path of tsv file to be loaded. 
            Eg: ""
    output : dataframe with tsv dataset loaded
    """
    # import pdb;pdb.set_trace()
    ## based on pandas version , one of the two syntaxes below should run
    print(filename)
    df = pd.read_csv(filename, sep='\t', index_col = False, on_bad_lines='skip')
    # df = pd.read_csv(filename, sep='\t', index_col = False, error_bad_lines='skip')
    print(df.shape)
    return df

In [10]:

def preprocess(sent):
    #converting all tokens to lowercase
    sent = str(sent).lower()
    #replace_acronyms
    sent = " ".join([COMMON_ABBREV[each] if each in COMMON_ABBREV else each for each in sent.split()])
    #replacing website links
    sent = re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", sent)
    #removing stopwords
    sent = remove_stopwords(sent)
    #stemming
    # sent = " ".join([ps.stem(word) for word in word_tokenize(sent)])
    #lemmatizing
    sent = " ".join([LEMMATIZER.lemmatize(word) for word in word_tokenize(sent)])
    #to filter out unicode or foreign characters
    sent = "".join([char for char in sent if char in LANGUAGE])
    #removing 's occurences
    sent = sent.replace("'s", "")
    #replacing words ending in "n't" with "word not"
    sent = sent.replace("wont", "will not").replace("won't", "will not").replace("cant", "can not")
    sent = sent.replace("can't", "can not")
    sent = re.sub(r"([a-zA-z])(n't)", r"\1 not", sent)
    sent = re.sub(r"(would|could|is|should)(nt)", r"\1 not", sent)
    #remove all HTML tags
    sent = re.sub(r"<.*?>", " ", sent)
    #replacing multiple punctuations with single occurrence
    sent = re.sub(r"\.+(\s?\.+)*", ".", sent)
    sent = re.sub(r"\!+", "!", sent)
    sent = re.sub(r"=+", "=", sent)
    sent = re.sub(r"_+", "_", sent)
    #introducing space after punctuation if not already there
    sent = re.sub(r"([,\.\"'\?\-:\(\)\\\/=\*\!_&#])(?!\s)", r"\1 ", sent)
    #introducing space before punctuation if not already there
    sent = re.sub(r"(?<!\s)([,\.\"'\?\-:\(\)\\\/=\!\*_])", r" \1", sent)
    #replacing words with more than 2 consecutive same character with two same characters
    sent = re.sub(r"([a-z])\1{1,}", r"\1\1", sent)
    #replacing multiple spaces with single space
    sent = re.sub(r"\s+", r" ", sent)
    
    return sent

In [27]:
def main(sample):
    """
    input : url to zip files of amazon customer reviews
            Eg: ""
    output : filename
             Eg:
    """
    # download_file = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv'
    # download_file = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz'
    # import pdb;pdb.set_trace()
    if sample:
        data = ['https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv']
    else: 
        data = [
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz',
            'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz'
            ]
    for url in data:
        print("URL >>>>   " + url)
        tsv_path = download_file(url)
        print("Loading dataset from >>> ", tsv_path)
        df = load_dataset(tsv_path)
        #removing rows with null in the following two columns
        print("BEFORE REMOVING NA IN review_body >>>>>")
        print(df.shape)
        df = df[df['review_body'].notna()]
        print("AFTER REMOVING NA IN review_body >>>>>")
        print(df.shape)
        df = df[df['review_headline'].notna()]
        print("AFTER REMOVING NA IN review_headline >>>>>")
        print(df.shape)
        tofile = tsv_path.replace('.tsv','_trim.tsv')
        df.to_csv(tofile, sep = '\t', index=False)
        print('created file >>  \t',tofile)

        #applying cleaning procedures to review_headline and review_body
        df['clean_review_headline'] = df.apply(lambda df: preprocess(df['review_headline']), axis=1)
        df['clean_review_body'] = df.apply(lambda df: preprocess(df['review_body']), axis=1)

        #Converting 5 star ratings to binary representation. 
        #Rating >=3 is positive i.e. 1
        #Rating <3 is negative i.e. 0

        df["Sentiment"] = df["star_rating"].apply(lambda score: "negative" if int(score) < 3 else "positive")
        df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

        tofile = tsv_path.replace('.tsv', '_clean.tsv')
        df.to_csv(tofile, sep = '\t', index=False)
        print('created file >>  \t',tofile)


# PRIMARY CELL TO TEST ABOVE FUNCTIONS

In [28]:
# set choice to 2 to run with real data
choice = 1
# print(args)
sample = False
if  choice == 1:
    sample = True
    print("Running with sample data")
    main(sample)
elif choice == 2:
    print("Running with real data")
    main(sample)
else:
    print('Choice not supported')

Running with sample data
URL >>>>   https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv
Zip File Already Downloaded and Decompressed
Loading dataset from >>>  sample_us.tsv
sample_us.tsv
(49, 15)
BEFORE REMOVING NA IN review_body >>>>>
(49, 15)
AFTER REMOVING NA IN review_body >>>>>
(49, 15)
AFTER REMOVING NA IN review_headline >>>>>
(49, 15)
created file >>  	 sample_us_trim.tsv
created file >>  	 sample_us_clean.tsv


In [29]:
df = pd.read_csv('sample_us_clean.tsv', sep='\t', index_col = False, on_bad_lines='skip')

In [30]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,clean_review_headline,clean_review_body,Sentiment
0,US,18778586,RDIJS7QYB6XNR,B00EDBY7X8,122952789,Monopoly Junior Board Game,Toys,5,0,0,N,Y,Five Stars,Excellent!!!,2015-08-31,star,excellent ! ! !,1
1,US,24769659,R36ED1U38IELG8,B00D7JFOPC,952062646,56 Pieces of Wooden Train Track Compatible wit...,Toys,5,0,0,N,Y,Good quality track at excellent price,Great quality wooden track (better than some o...,2015-08-31,good quality track excellent price,great quality wooden track ( better tried ) . ...,1
2,US,44331596,R1UE3RPRGCOLD,B002LHA74O,818126353,Super Jumbo Playing Cards by S&S Worldwide,Toys,2,1,1,N,Y,Two Stars,Cards are not as big as pictured.,2015-08-31,star,card big pictured .,0
3,US,23310293,R298788GS6I901,B00ARPLCGY,261944918,Barbie Doll and Fashions Barbie Gift Set,Toys,5,0,0,N,Y,my daughter loved it and i liked the price and...,my daughter loved it and i liked the price and...,2015-08-31,daughter loved liked price came .,daughter loved liked price came shopping ton p...,1
4,US,38745832,RNX4EXOBBPN5,B00UZOPOFW,717410439,Emazing Lights eLite Flow Glow Sticks - Spinni...,Toys,1,1,1,N,Y,DONT BUY THESE!,Do not buy these! They break very fast I spun ...,2015-08-31,dont buy these !,buy these ! break fast spun minute end flew do...,0
